In [1]:
!pip3 install tensorflow==2.0.0a0

     |████████████████████████████████| 80.3 MB 67 kB/s 
     |████████████████████████████████| 411 kB 34.5 MB/s 
     |████████████████████████████████| 50 kB 5.0 MB/s 
     |████████████████████████████████| 3.0 MB 40.6 MB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adadelta
from sklearn.datasets import load_files
from skmultilearn.model_selection import iterative_train_test_split

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [4]:
tf.__version__

'2.0.0-alpha0'

In [5]:
df_train = pd.read_csv('../input/garbage-classification/one-indexed-files-notrash_train.txt', header=None)
df_test = pd.read_csv('../input/garbage-classification/one-indexed-files-notrash_test.txt', header=None)
df_val = pd.read_csv('../input/garbage-classification/one-indexed-files-notrash_val.txt', header=None)

df_train.columns = df_val.columns = df_test.columns=["temp"]

In [6]:
def getNamesTargets(df):
    files = []
    targets = []
    path='/kaggle/input/garbage-classification/garbage classification/Garbage classification/'

    for i in range(len(df)):
        f, n = df.iloc[i, 0].split(" ")
        if n=='1':
            fil=path+'glass/'+f
        elif n=='2':
            fil=path+'paper/'+f
        elif n=='3':
            fil=path+'cardboard/'+f
        elif n=='4':
            fil=path+'plastic/'+f
        elif n=='5':
            fil=path+'metal/'+f
        elif n=='6':
            fil=path+'trash/'+f
        files.append(fil)
        targets.append(n)
    return files, targets

In [7]:
df_train["filename"], df_train["target"] = getNamesTargets(df_train)
df_test["filename"], df_test["target"] = getNamesTargets(df_test)
df_val["filename"], df_val["target"] = getNamesTargets(df_val)

df_train.drop(columns=["temp"], inplace=True)
df_test.drop(columns=["temp"], inplace=True)
df_val.drop(columns=["temp"], inplace=True)

df_train.head()

,filename,target
0,/kaggle/input/garbage-classification/garbage c...,3
1,/kaggle/input/garbage-classification/garbage c...,2
2,/kaggle/input/garbage-classification/garbage c...,2
3,/kaggle/input/garbage-classification/garbage c...,1
4,/kaggle/input/garbage-classification/garbage c...,1


In [8]:
df_test.head()

,filename,target
0,/kaggle/input/garbage-classification/garbage c...,2
1,/kaggle/input/garbage-classification/garbage c...,2
2,/kaggle/input/garbage-classification/garbage c...,3
3,/kaggle/input/garbage-classification/garbage c...,1
4,/kaggle/input/garbage-classification/garbage c...,2


In [9]:
df_val.head()

,filename,target
0,/kaggle/input/garbage-classification/garbage c...,3
1,/kaggle/input/garbage-classification/garbage c...,4
2,/kaggle/input/garbage-classification/garbage c...,1
3,/kaggle/input/garbage-classification/garbage c...,1
4,/kaggle/input/garbage-classification/garbage c...,1


In [10]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )


test_datagen = ImageDataGenerator(rescale = 1./255)

val_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
train = train_datagen.flow_from_dataframe(dataframe=df_train,
                                        x_col="filename", y_col="target",
                                        target_size=(64, 64), class_mode="categorical", batch_size=32,
                                        shuffle=True, seed=123, validate_filenames=True)



test = test_datagen.flow_from_dataframe(dataframe=df_test,
                                      x_col="filename", y_col="target", target_size = (64, 64),
                                      batch_size = 32, class_mode = 'categorical',shuffle=True, seed=123,
                                       validate_filenames=True)

val = val_datagen.flow_from_dataframe(dataframe=df_val,
                                      x_col="filename", y_col="target", target_size = (64, 64),
                                      batch_size = 32, class_mode = 'categorical',shuffle=True, seed=123,
                                       validate_filenames=True)

Found 1768 validated image filenames belonging to 6 classes.
Found 431 validated image filenames belonging to 6 classes.
Found 328 validated image filenames belonging to 6 classes.


In [12]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 strides=2,
                 input_shape=(64,64, 3),
                 activation ='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(32, kernel_size=(3, 3), strides=2, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Conv2D(32, kernel_size=(3, 3), strides=2, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 6, activation = 'sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [14]:
cd = [es, mc]

In [15]:
model.fit(train,
    epochs=1000,
    callbacks=cd,
    validation_data=test)

Epoch 1/1000
56/56 [==============================] - 27s 475ms/step - loss: 1.7308 - accuracy: 0.2223 - val_loss: 1.7343 - val_accuracy: 0.2088
Epoch 2/1000
56/56 [==============================] - 9s 169ms/step - loss: 1.6865 - accuracy: 0.2528 - val_loss: 1.6343 - val_accuracy: 0.3387
Epoch 3/1000
56/56 [==============================] - 10s 176ms/step - loss: 1.6021 - accuracy: 0.3218 - val_loss: 1.5643 - val_accuracy: 0.3271
Epoch 4/1000
56/56 [==============================] - 10s 172ms/step - loss: 1.5571 - accuracy: 0.3309 - val_loss: 1.5119 - val_accuracy: 0.3643
Epoch 5/1000
56/56 [==============================] - 10s 173ms/step - loss: 1.5710 - accuracy: 0.3083 - val_loss: 1.5065 - val_accuracy: 0.3364
Epoch 6/1000
56/56 [==============================] - 10s 178ms/step - loss: 1.5165 - accuracy: 0.3196 - val_loss: 1.4944 - val_accuracy: 0.3689
Epoch 7/1000
56/56 [==============================] - 10s 172ms/step - loss: 1.5186 - accuracy: 0.3207 - val_loss: 1.4639 - val_acc

In [16]:
score=model.evaluate(val)

print(model.metrics_names)

for i in score:
    print(i)

11/11 [==============================] - 4s 348ms/step - loss: nan - accuracy: 0.1982
['loss', 'accuracy']
nan
0.19817074
